##Git Clone##

In [ ]:
!git clone https://github.com/KaleF07/WasteClassification.git

Cloning into 'WasteClassification'...
remote: Enumerating objects: 24766, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 24766 (delta 0), reused 6 (delta 0), pack-reused 24760
Receiving objects: 100% (24766/24766), 207.36 MiB | 21.51 MiB/s, done.
Updating files: 100% (25079/25079), done.


##INSTALLATIONS##

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

##PATH##

In [ ]:
export_path = Path('/content/WasteClassification/DATASET')
export_path.ls()

In [ ]:
path = export_path/'TRAIN'
get_image_files(path)

##NUMBER OF IMAGES##

In [ ]:
import os

lst = os.listdir('/content/WasteClassification/DATASET/TRAIN/O') # your directory path
number_files_Organic = len(lst)
print (number_files_Organic)

In [ ]:
import os

lst = os.listdir('/content/WasteClassification/DATASET/TRAIN/R') # your directory path
number_files_Recyclable = len(lst)
print (number_files_Recyclable)

##DATA LOADING & AUGMENTATION##

In [ ]:
waste = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #image->discrete category
    get_items=get_image_files, # path->images
    splitter=RandomSplitter(valid_pct=0.2, seed=42), #how to train-valid split
    get_y=parent_label, # get the label from the directory name
    item_tfms=Resize(128),
    batch_tfms=[*aug_transforms(mult=1,flip_vert=True,max_zoom=2,max_warp=0,do_flip=True),
                Brightness (max_lighting:float=0.2, p:float=0.75,
             draw:float|MutableSequence|callable=None, batch=False),
                 grayscale (x),
                cutout_gaussian (x:torch.Tensor, areas:list),
                CropPad (size:int|tuple, pad_mode:PadMode='zeros', **kwargs)) # resize to 128x128

dls = waste.dataloaders(path)

In [ ]:
dls.train.show_batch(max_n=50, nrows=6)

In [ ]:
dls.valid.show_batch(max_n=50, nrows=6)